# This kernel use simple transfer learning to detect ship exist or not
*** For biginner, can get about 85% accuracy train on 10 epoch (~20min one epoch no GPU, ~100s one epoch have GPU)**
* Use ResNet50 to do transfer learning 
* Load 5000 picture to be training data 
* Split 4000 training set , 1000 validate set  (0.2%)
* Image size 256 x 256, RGB data
* Using ImageGenerator to do data augumatation

## Load segmentation file

In [1]:
import os
import gc
print(os.listdir("../input"))
import numpy as np 
import pandas as pd
import time

['sample_submission_v2.csv', 'test_v2', 'train_v2', 'train_ship_segmentations_v2.csv']


In [2]:
train = pd.read_csv('../input/train_ship_segmentations_v2.csv')

In [3]:
train.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


## Tranfer EncodedPixels to target 
* have ship ==> 1
* No ship ==> 0

In [4]:
train['exist_ship'] = train['EncodedPixels'].fillna(0)
train.loc[train['exist_ship']!=0,'exist_ship']=1
del train['EncodedPixels']

## We found there are some duplicate image in training data
* groupby duplicate image 

In [5]:
print(len(train['ImageId']))
print(train['ImageId'].value_counts().shape[0])
train_gp = train.groupby('ImageId').sum().reset_index()
train_gp.loc[train_gp['exist_ship']>0,'exist_ship']=1

231723
192556


## Balance have chip and no chip data
* Remove 100000 data of no chip

In [6]:
print(train_gp['exist_ship'].value_counts())
train_gp= train_gp.sort_values(by='exist_ship')
train_gp = train_gp.drop(train_gp.index[0:100000])

0    150000
1     42556
Name: exist_ship, dtype: int64


## Set Training set count
* prevent large data cause much time 

In [7]:
print(train_gp['exist_ship'].value_counts())
train_sample = train_gp.sample(5000)
print(train_sample['exist_ship'].value_counts())
print (train_sample.shape)

0    50000
1    42556
Name: exist_ship, dtype: int64
0    2710
1    2290
Name: exist_ship, dtype: int64
(5000, 2)


## Load training data function
* load training data to numpy array for training 

In [8]:
Train_path = '../input/train_v2/'
Test_path = '../input/test_v2/'

In [9]:
%%time
training_img_data = []
target_data = []
from PIL import Image
data = np.empty((len(train_sample['ImageId']),256, 256,3), dtype=np.uint8)
data_target = np.empty((len(train_sample['ImageId'])), dtype=np.uint8)
image_name_list = os.listdir(Train_path)
index = 0
for image_name in image_name_list:
    if image_name in list(train_sample['ImageId']):
        imageA = Image.open(Train_path+image_name).resize((256,256)).convert('RGB')
        data[index]=imageA
        data_target[index]=train_sample[train_gp['ImageId'].str.contains(image_name)]['exist_ship'].iloc[0]
        index+=1
        
print(data.shape)
print(data_target.shape)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


(5000, 256, 256, 3)
(5000,)
CPU times: user 6min 47s, sys: 1.56 s, total: 6min 49s
Wall time: 7min 4s


## Doing One hot on target
* Set target to one hot target for classification problem

In [10]:
from sklearn.preprocessing import OneHotEncoder
targets =data_target.reshape(len(data_target),-1)
enc = OneHotEncoder()
enc.fit(targets)
targets = enc.transform(targets).toarray()
print(targets.shape)

(5000, 2)


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Split Training data to training data and validate data to detect overfit 

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data,targets, test_size = 0.2)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((4000, 256, 256, 3), (1000, 256, 256, 3), (4000, 2), (1000, 2))

## Data augumatation
* Using ImageDataGenerator

In [12]:
from keras.preprocessing.image import ImageDataGenerator
img_gen = ImageDataGenerator(
    rescale=1./255,
    zca_whitening = False,
    rotation_range = 90,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    brightness_range = [0.5, 1.5],
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True
    
)

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

## Load ResNet50 model with Keras
* on Kaggle kernel, please turn on the internet setting to Internet connect  on right window

In [13]:
#from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
#from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
from keras.applications.resnet50 import ResNet50 as ResModel
#from keras.applications.vgg16 import VGG16 as VGG16Model
img_width, img_height = 256, 256
model = ResModel(weights = 'imagenet', include_top=False, input_shape = (img_width, img_height, 3))

/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


## Add fully connect layer
* Freeze convolution layer and add fully connect layer
* On this case, we only need predict 2 category (1. have ship, 2. no ship)
* For transfer learning, we only need to train parametric on fully connect layer

In [14]:
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model 
for layer in model.layers:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


##  Set Hyperparameter and Start training 
* SGD optimizer
* Using categorical_crossentropy to be loss function
* lrate set to 0.001 (Maybe we have better value, In here, I have no experence on this)

In [15]:
from keras import optimizers
epochs = 10
lrate = 0.001
decay = lrate/epochs
#adam = optimizers.Adam(lr=lrate,beta_1=0.9, beta_2=0.999, decay=decay)
sgd = optimizers.SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model_final.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [16]:
model_final.fit_generator(img_gen.flow(x_train, y_train, batch_size = 16),steps_per_epoch = len(x_train)/16, validation_data = (x_val,y_val), epochs = epochs )
model_final.save('ResNet_transfer_ship.h5')

Epoch 1/10
250/250 [==============================] - 69s 276ms/step - loss: 1.1393 - acc: 0.7250 - val_loss: 0.3883 - val_acc: 0.8310
Epoch 2/10
250/250 [==============================] - 62s 248ms/step - loss: 0.4802 - acc: 0.7870 - val_loss: 0.3376 - val_acc: 0.8600
Epoch 3/10
250/250 [==============================] - 61s 246ms/step - loss: 0.4331 - acc: 0.8048 - val_loss: 0.3392 - val_acc: 0.8670
Epoch 4/10
250/250 [==============================] - 61s 245ms/step - loss: 0.4186 - acc: 0.8113 - val_loss: 0.3146 - val_acc: 0.8750
Epoch 5/10
250/250 [==============================] - 63s 251ms/step - loss: 0.4076 - acc: 0.8243 - val_loss: 0.3186 - val_acc: 0.8850
Epoch 6/10
250/250 [==============================] - 64s 255ms/step - loss: 0.4120 - acc: 0.8135 - val_loss: 0.3023 - val_acc: 0.8660
Epoch 7/10
250/250 [==============================] - 61s 245ms/step - loss: 0.4017 - acc: 0.8225 - val_loss: 0.3098 - val_acc: 0.8690
Epoch 8/10
250/250 [==============================] - 6

In [17]:
gc.collect()

5

## Predict accuracy by random read training data

* Get random 2000 data from training set

In [18]:
train_predict_sample = train_gp.sample(2000)
print(train_predict_sample['exist_ship'].value_counts())

0    1131
1     869
Name: exist_ship, dtype: int64


* Load predict data 

In [19]:
%%time
from PIL import Image
data_predict = np.empty((len(train_predict_sample['ImageId']),256, 256,3), dtype=np.uint8)
data_target_predict = np.empty((len(train_predict_sample['ImageId'])), dtype=np.uint8)
image_name_list = os.listdir(Train_path)
index = 0
for image_name in image_name_list:
    if image_name in list(train_predict_sample['ImageId']):
        imageA = Image.open(Train_path+image_name).resize((256,256)).convert('RGB')
        data_predict[index]=imageA
        data_target_predict[index]=train_predict_sample[train_gp['ImageId'].str.contains(image_name)]['exist_ship'].iloc[0]
        index+=1
        
print(data_predict.shape)
print(data_target_predict.shape)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


(2000, 256, 256, 3)
(2000,)
CPU times: user 2min 32s, sys: 552 ms, total: 2min 33s
Wall time: 2min 38s


* Do one hot for predict target

In [20]:
from sklearn.preprocessing import OneHotEncoder
targets_predict =data_target_predict.reshape(len(data_target_predict),-1)
enc = OneHotEncoder()
enc.fit(targets_predict)
targets_predict = enc.transform(targets_predict).toarray()
print(targets_predict.shape)

(2000, 2)


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


* Evaluate predict

In [21]:
predict_ship = model_final.evaluate(data_predict,targets_predict)

2000/2000 [==============================] - 7s 3ms/step


* Result

In [22]:
print ('Accuracy of random data = '+ str(round(predict_ship[1]*100)) + "%")

Accuracy of random data = 88.0%


In [23]:
image_test_name_list = os.listdir(Test_path)
data_test = np.empty((len(image_test_name_list),256, 256,3), dtype=np.uint8)
test_name = []
index = 0
for image_name in image_test_name_list:
    imageA = Image.open(Test_path+image_name).resize((256,256)).convert('RGB')
    test_name.append(image_name)
    data_test[index]=imageA
    index+=1
print (data_test.shape)

(15606, 256, 256, 3)


In [24]:
result = model_final.predict(data_test)

In [25]:
result_list={
    "ImageId": test_name,
    "Have_ship":np.argmax(result,axis=1)
}
result_pd = pd.DataFrame(result_list)
result_pd.to_csv('Have_ship_or_not.csv',index = False)

## Conclution
*  We can use tranfer learning to detect ship or not , and get higher accuracy on it 
*  If we get 95% accuracy up, we can merge it with Unet model to produce a final submission
*  Like Iafoss kernel: https://www.kaggle.com/iafoss/fine-tuning-resnet34-on-ship-detection/notebook